## 1. Environment Setup

In [81]:
# Environment setup
from pathlib import Path
from typing import Optional
import sys

# Helper to find project root
def _find_root(start: Optional[Path] = None) -> Path:
    p = start or Path.cwd()
    for _ in range(6):
        if (p / 'data').exists() or (p / '.git').exists() or (p / 'notebooks').exists():
            return p
        p = p.parent
    return Path.cwd()

# Resolve project directories consistently
ROOT = _find_root()
DATA_DIR = ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
INTERIM_DIR = DATA_DIR / 'interim'
PROCESSED_DIR = DATA_DIR / 'processed'
MATCHES_DIR = DATA_DIR / 'matches'
STANDINGS_DIR = INTERIM_DIR / 'scraped_standings'
FIG_DIR = ROOT / 'reports' / 'figures'

# Create directories
for d in [RAW_DIR, INTERIM_DIR, PROCESSED_DIR, MATCHES_DIR, FIG_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"🎯 Environment setup complete")
print(f"   ROOT: {ROOT}")
print(f"   MATCHES_DIR: {MATCHES_DIR}")
print(f"   STANDINGS_DIR: {STANDINGS_DIR}")

🎯 Environment setup complete
   ROOT: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks
   MATCHES_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\matches
   STANDINGS_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\interim\scraped_standings


## 2. Import Libraries

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


## 3. Load Match Data

נטען את כל נתוני המשחקים של העונות הרגילות מטרנספרמרקט.

In [83]:
# Load all matches data
matches_file = MATCHES_DIR / 'matches_all_seasons_ligat_haal_transfermarkt.csv'

print(f"📥 Loading matches data from: {matches_file}")
df_matches = pd.read_csv(matches_file)

print(f"✅ Loaded {len(df_matches)} matches")
print(f"   Seasons: {df_matches['season'].min()} to {df_matches['season'].max()}")
print(f"   Unique seasons: {df_matches['season'].nunique()}")

print("\n🔍 Sample data:")
display(df_matches.head(10))

print("\n📊 Data structure:")
print(df_matches.info())

📥 Loading matches data from: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\matches\matches_all_seasons_ligat_haal_transfermarkt.csv
✅ Loaded 3754 matches
   Seasons: 2006/07 to 2025/26
   Unique seasons: 20

🔍 Sample data:


,round,home,score,away,season,season_year
0,1,H. Kfar Saba,4:1,H. Petah Tikva,2006/07,2006
1,2,M. Petah Tikva,0:0,Hakoah Amidar,2006/07,2006
2,3,FC Ashdod,1:0,Maccabi Herzlya,2006/07,2006
3,4,Maccabi Netanya,3:1,Maccabi Haifa,2006/07,2006
4,5,M. Tel Aviv,1:2,B. Jerusalem,2006/07,2006
5,6,Bnei Yehuda,1:1,Hapoel Tel Aviv,2006/07,2006
6,7,Hakoah Amidar,3:2,Bnei Yehuda,2006/07,2006
7,8,Maccabi Herzlya,0:3,M. Petah Tikva,2006/07,2006
8,9,Maccabi Haifa,1:0,FC Ashdod,2006/07,2006
9,10,H. Petah Tikva,0:0,Maccabi Netanya,2006/07,2006



📊 Data structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3754 entries, 0 to 3753
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   round        3754 non-null   int64 
 1   home         3754 non-null   object
 2   score        3754 non-null   object
 3   away         3754 non-null   object
 4   season       3754 non-null   object
 5   season_year  3754 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 176.1+ KB
None


In [84]:
# Parse score column and calculate goal difference
def parse_score(score_str):
    """Parse score string like '4:1' into home and away goals."""
    try:
        if pd.isna(score_str) or score_str == '' or ':' not in str(score_str):
            return None, None
        home, away = str(score_str).strip().split(':')
        return int(home), int(away)
    except:
        return None, None

# Apply parsing
df_matches[['home_goals', 'away_goals']] = df_matches['score'].apply(
    lambda x: pd.Series(parse_score(x))
)

# Calculate goal difference
df_matches['goal_diff'] = abs(df_matches['home_goals'] - df_matches['away_goals'])

# Remove matches without valid scores
df_matches = df_matches.dropna(subset=['home_goals', 'away_goals'])

# Filter matches with 3+ goal difference
df_big_margin = df_matches[df_matches['goal_diff'] >= 3].copy()

print(f"✅ Parsed {len(df_matches)} matches")
print(f"\n🎯 Matches with 3+ goal difference:")
print(f"   Total: {len(df_big_margin)} matches")
print(f"   Percentage: {len(df_big_margin) / len(df_matches) * 100:.2f}% of all matches")

print("\n📊 Distribution by season:")
summary = df_matches.groupby('season').agg(
    total_matches=('round', 'count'),
    big_margin=('goal_diff', lambda x: (x >= 3).sum())
).copy()
summary['pct'] = (summary['big_margin'] / summary['total_matches'] * 100).round(2)
display(summary)

print("\n🔍 Examples of big margin matches:")
display(df_big_margin[['season', 'round', 'home', 'score', 'away', 'goal_diff']].head(10))

✅ Parsed 3754 matches

🎯 Matches with 3+ goal difference:
   Total: 519 matches
   Percentage: 13.83% of all matches

📊 Distribution by season:


,total_matches,big_margin,pct
season,,,
2006/07,198,23,11.62
2007/08,198,26,13.13
2008/09,198,17,8.59
2009/10,240,38,15.83
2010/11,240,37,15.42
2011/12,240,34,14.17
2012/13,182,27,14.84
2013/14,182,21,11.54
2014/15,182,23,12.64



🔍 Examples of big margin matches:


,season,round,home,score,away,goal_diff
0,2006/07,1,H. Kfar Saba,4:1,H. Petah Tikva,3
7,2006/07,8,Maccabi Herzlya,0:3,M. Petah Tikva,3
17,2006/07,18,Hapoel Tel Aviv,4:1,Hakoah Amidar,3
27,2006/07,28,FC Ashdod,3:0,H. Kfar Saba,3
30,2006/07,31,M. Tel Aviv,4:0,Bnei Yehuda,4
32,2006/07,33,H. Kfar Saba,3:0,M. Petah Tikva,3
33,2006/07,34,B. Jerusalem,3:0,Maccabi Herzlya,3
39,2006/07,40,Hakoah Amidar,0:3,H. Petah Tikva,3
40,2006/07,41,FC Ashdod,1:4,B. Jerusalem,3
102,2006/07,103,Maccabi Haifa,0:4,Maccabi Herzlya,4


## 4. ניתוח משחקים חד-צדדיים

נבדוק כמה משחקים נגמרו בהפרש של 3+ שערים (משחקים חד-צדדיים).
נסווג אותם לפי מיקום הקבוצות בטבלה בזמן המשחק.

## 5. סקרייפינג טבלאות מיקומים

נסקרייף מטרנספרמרקט את מיקום הקבוצות בכל מחזור בכל עונה.
נבנה טבלה לכל עונה: שורות = מחזורים, עמודות = קבוצות, ערכים = מיקום.

In [87]:
import requests
from bs4 import BeautifulSoup
import time
import random

def scrape_tm_standings(season_str, round_num):
    """
    Scrape standings table from Transfermarkt for a specific round.
    
    Args:
        season_str: Season like "2006/07"
        round_num: Round number
    
    Returns:
        Dictionary: {team_name: position} or None if failed
    """
    year = int(season_str.split('/')[0])
    url = f"https://www.transfermarkt.com/ligat-haal/spieltagtabelle/wettbewerb/ISR1?saison_id={year}&spieltag={round_num}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the table - try multiple approaches
        table = soup.find('table', {'class': 'items'})
        
        # If not found, try finding any table with "TABLE LIGAT" text nearby
        if not table:
            # Look for all tables and find one with team data
            tables = soup.find_all('table')
            for t in tables:
                rows = t.find_all('tr')
                if len(rows) > 10:  # Should have at least 12-16 teams
                    table = t
                    break
        
        if not table:
            print(f"         DEBUG: No table found. Page HTML length: {len(response.content)}")
            return None
        
        # Try to find tbody, if not just use the table
        tbody = table.find('tbody')
        rows = tbody.find_all('tr') if tbody else table.find_all('tr')
        
        positions = {}
        
        for row in rows:
            cells = row.find_all('td')
            if len(cells) < 3:
                continue
            
            try:
                # Position is usually in first cell
                rank_text = cells[0].text.strip()
                if not rank_text.isdigit():
                    continue
                rank = int(rank_text)
                
                # Team name is usually in 2nd or 3rd cell
                # Try cell 2 first, then cell 1
                team = cells[2].text.strip() if len(cells) > 2 else cells[1].text.strip()
                
                if team and rank:
                    positions[team] = rank
            except:
                continue
        
        return positions if positions else None
        
    except Exception as e:
        print(f"         DEBUG: Exception - {str(e)}")
        return None


def scrape_season_positions(season_str, expected_rounds):
    """
    Scrape all rounds for a season.
    
    Args:
        season_str: Season like "2006/07"
        expected_rounds: Expected number of rounds (33, 30, or 26)
    
    Returns:
        DataFrame with rounds as index, teams as columns, positions as values
    """
    print(f"\n   🔨 Scraping {season_str} (expecting {expected_rounds} rounds)...")
    
    position_data = []
    
    for round_num in range(1, expected_rounds + 1):
        positions = scrape_tm_standings(season_str, round_num)
        
        if positions:
            round_data = {'round': round_num}
            round_data.update(positions)
            position_data.append(round_data)
            print(f"      ✅ Round {round_num}: {len(positions)} teams")
        else:
            print(f"      ❌ FAILED Round {round_num}")
            # Stop here and let user check what's wrong
            print(f"\n⚠️⚠️⚠️ STOPPED AT ROUND {round_num} - Check what's wrong!")
            print(f"      URL: https://www.transfermarkt.com/ligat-haal/spieltagtabelle/wettbewerb/ISR1?saison_id={int(season_str.split('/')[0])}&spieltag={round_num}")
            return None
        
        # Be nice to the server
        time.sleep(random.uniform(2, 4))
    
    if not position_data:
        return None
    
    # Convert to DataFrame
    df_positions = pd.DataFrame(position_data)
    df_positions = df_positions.set_index('round')
    
    return df_positions


print("🏗️ Building position tables from Transfermarkt")
print("=" * 70)
print("מבנה העונות:")
print("  2006/07 - 2008/09: 12 קבוצות, 33 מחזורים")
print("  2009/10 - 2011/12: 16 קבוצות, 30 מחזורים")
print("  2012/13 ואילך: 14 קבוצות, 26 מחזורים")
print("=" * 70)

🏗️ Building position tables from Transfermarkt
מבנה העונות:
  2006/07 - 2008/09: 12 קבוצות, 33 מחזורים
  2009/10 - 2011/12: 16 קבוצות, 30 מחזורים
  2012/13 ואילך: 14 קבוצות, 26 מחזורים


### בדיקה - עונה אחת

נבדוק עונה אחת קודם כדי לוודא שהסקרייפינג עובד כמו שצריך.

In [88]:
# Test with season 2009/10 (16 teams, 30 rounds)
test_season = '2009/10'
test_df = scrape_season_positions(test_season, expected_rounds=30)

if test_df is not None:
    print(f"\n✅ SUCCESS! Got {len(test_df)} rounds")
    print(f"   Teams: {len(test_df.columns)}")
    print(f"\n   Last round:")
    display(test_df.iloc[-1].sort_values())
else:
    print("\n❌ Failed - check the URL and see what's wrong")


   🔨 Scraping 2009/10 (expecting 30 rounds)...
      ✅ Round 1: 16 teams
      ✅ Round 2: 16 teams
      ✅ Round 3: 16 teams
      ✅ Round 4: 16 teams
      ✅ Round 5: 16 teams
      ✅ Round 6: 16 teams
      ✅ Round 7: 16 teams
      ✅ Round 8: 16 teams
      ✅ Round 9: 16 teams
      ✅ Round 10: 16 teams
      ✅ Round 11: 16 teams
      ✅ Round 12: 16 teams
      ✅ Round 13: 16 teams
      ✅ Round 14: 16 teams
      ✅ Round 15: 16 teams
      ✅ Round 16: 16 teams
      ✅ Round 17: 16 teams
      ✅ Round 18: 16 teams
      ✅ Round 19: 16 teams
      ✅ Round 20: 16 teams
      ✅ Round 21: 16 teams
      ✅ Round 22: 16 teams
      ✅ Round 23: 16 teams
      ✅ Round 24: 16 teams
      ✅ Round 25: 16 teams
      ✅ Round 26: 16 teams
      ✅ Round 27: 16 teams
      ✅ Round 28: 16 teams
      ✅ Round 29: 16 teams
      ✅ Round 30: 16 teams

✅ SUCCESS! Got 30 rounds
   Teams: 16

   Last round:


Maccabi Haifa       1
Hapoel Tel Aviv     2
M. Tel Aviv         3
B. Jerusalem        4
Bnei Yehuda         5
FC Ashdod           6
Bnei Sakhnin        7
H. Beer Sheva       8
Maccabi Netanya     9
M. Petah Tikva     10
H. Ramat Gan       11
Hapoel Haifa       12
H. Petah Tikva     13
Hapoel Acre        14
M. Ahi Nazareth    15
Hapoel Raanana     16
Name: 30, dtype: int64

### סקרייפינג כל העונות

עכשיו נריץ על כל העונות עם מנגנון caching - אם הקובץ כבר קיים, נטען אותו במקום לסקרוף מחדש.

In [89]:
# Create cache directory
POSITION_CACHE_DIR = PROCESSED_DIR / 'position_tables_by_round_tm'
POSITION_CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Define season structure
def get_season_structure(season_str):
    """Get expected number of rounds for a season."""
    year = int(season_str.split('/')[0])
    
    if year <= 2008:
        return 33  # 12 teams, 33 rounds
    elif year <= 2011:
        return 30  # 16 teams, 30 rounds
    else:
        return 26  # 14 teams, 26 rounds

# Get all seasons
all_seasons = sorted(df_matches['season'].unique())

print("🚀 Scraping position tables for all seasons")
print("=" * 70)
print(f"Total seasons to process: {len(all_seasons)}")
print("=" * 70)

position_tables = {}
failed_seasons = []

for i, season in enumerate(all_seasons, 1):
    print(f"\n[{i}/{len(all_seasons)}] Processing {season}...")
    
    # Check cache first
    season_safe = season.replace('/', '_')
    cache_file = POSITION_CACHE_DIR / f'positions_by_round_{season_safe}_tm.csv'
    
    if cache_file.exists():
        print(f"   📦 Loading from cache...")
        df_positions = pd.read_csv(cache_file, index_col=0)
        position_tables[season] = df_positions
        print(f"   ✅ Loaded {len(df_positions)} rounds, {len(df_positions.columns)} teams")
    else:
        # Scrape
        expected_rounds = get_season_structure(season)
        df_positions = scrape_season_positions(season, expected_rounds)
        
        if df_positions is not None:
            # Save to cache
            df_positions.to_csv(cache_file, encoding='utf-8-sig')
            position_tables[season] = df_positions
            print(f"   ✅ Scraped and cached: {len(df_positions)} rounds, {len(df_positions.columns)} teams")
        else:
            failed_seasons.append(season)
            print(f"   ❌ FAILED - stopping here")
            break

print("\n" + "=" * 70)
if failed_seasons:
    print(f"⚠️ Failed seasons: {failed_seasons}")
    print("Please check what went wrong before continuing.")
else:
    print(f"✅ SUCCESS! All {len(position_tables)} seasons scraped")
    print(f"📁 Cached in: {POSITION_CACHE_DIR}")
    
    # Show summary
    print("\n📊 Summary:")
    for season in sorted(position_tables.keys()):
        df = position_tables[season]
        print(f"   {season}: {len(df)} rounds, {len(df.columns)} teams")

print("=" * 70)

🚀 Scraping position tables for all seasons
Total seasons to process: 20

[1/20] Processing 2006/07...

   🔨 Scraping 2006/07 (expecting 33 rounds)...
      ✅ Round 1: 12 teams
      ✅ Round 2: 12 teams
      ✅ Round 3: 12 teams
      ✅ Round 4: 12 teams
      ✅ Round 5: 12 teams
      ✅ Round 6: 12 teams
      ✅ Round 7: 12 teams
      ✅ Round 8: 12 teams
      ✅ Round 9: 12 teams
      ✅ Round 10: 12 teams
      ✅ Round 11: 12 teams
      ✅ Round 12: 12 teams
      ✅ Round 13: 12 teams
      ✅ Round 14: 12 teams
      ✅ Round 15: 12 teams
      ✅ Round 16: 12 teams
      ✅ Round 17: 12 teams
      ✅ Round 18: 12 teams
      ✅ Round 19: 12 teams
      ✅ Round 20: 12 teams
      ✅ Round 21: 12 teams
      ✅ Round 22: 12 teams
      ✅ Round 23: 12 teams
      ✅ Round 24: 12 teams
      ✅ Round 25: 12 teams
      ✅ Round 26: 12 teams
      ✅ Round 27: 12 teams
      ✅ Round 28: 12 teams
      ✅ Round 29: 12 teams
      ✅ Round 30: 12 teams
      ✅ Round 31: 12 teams
      ✅ Round 32: 12 t

In [90]:
# בדיקה - האם הסקרייפינג הצליח?
print(f"✅ סה\"כ עונות שנסקרפו: {len(position_tables)}")
print(f"⚠️ עונות שנכשלו: {len(failed_seasons)}")

if failed_seasons:
    print(f"\nעונות שנכשלו: {failed_seasons}")
else:
    print("\n🎉 כל העונות נסקרפו בהצלחה!")
    print("\n📊 סיכום:")
    for season in sorted(position_tables.keys())[:5]:
        df = position_tables[season]
        print(f"   {season}: {len(df)} מחזורים, {len(df.columns)} קבוצות")

✅ סה"כ עונות שנסקרפו: 20
⚠️ עונות שנכשלו: 0

🎉 כל העונות נסקרפו בהצלחה!

📊 סיכום:
   2006/07: 33 מחזורים, 12 קבוצות
   2007/08: 33 מחזורים, 12 קבוצות
   2008/09: 33 מחזורים, 12 קבוצות
   2009/10: 30 מחזורים, 16 קבוצות
   2010/11: 30 מחזורים, 16 קבוצות


## 6. סיווג משחקים לפי מיקום הקבוצות

עכשיו נסווג כל משחק בהפרש גדול לפי מיקום הקבוצות בטבלה **לפני** המשחק:
- **צמרת (Top)**: מקומות 1-4
- **אמצע (Middle)**: מקומות 5-10
- **תחתית (Bottom)**: מקומות 11+

In [91]:
def get_team_tier(position):
    """Classify team tier based on league position."""
    if position <= 4:
        return 'Top'
    elif position <= 10:
        return 'Middle'
    else:
        return 'Bottom'


print("🔍 Classifying big margin matches by team positions...")
print("=" * 70)

results = []
skipped = []

for _, match in df_big_margin.iterrows():
    season = match['season']
    match_round = match['round']
    home_team = match['home']
    away_team = match['away']
    
    # Check if we have position data for this season
    if season not in position_tables:
        skipped.append({'season': season, 'reason': 'No position data'})
        continue
    
    positions_df = position_tables[season]
    
    # Find position BEFORE this match
    # If round 1, use round 1 positions (start of season)
    # Otherwise, use previous round positions
    lookup_round = 1 if match_round == 1 else match_round - 1
    
    # Check if this round exists
    if lookup_round not in positions_df.index:
        skipped.append({
            'season': season, 
            'round': match_round, 
            'reason': f'Round {lookup_round} not in data'
        })
        continue
    
    # Get positions for this round
    round_positions = positions_df.loc[lookup_round]
    
    # Check if both teams exist in the data
    if home_team not in round_positions:
        skipped.append({
            'season': season,
            'round': match_round,
            'home': home_team,
            'reason': f'Home team "{home_team}" not found'
        })
        continue
    
    if away_team not in round_positions:
        skipped.append({
            'season': season,
            'round': match_round,
            'away': away_team,
            'reason': f'Away team "{away_team}" not found'
        })
        continue
    
    # Get positions
    home_position = int(round_positions[home_team])
    away_position = int(round_positions[away_team])
    
    # Get tiers
    home_tier = get_team_tier(home_position)
    away_tier = get_team_tier(away_position)
    
    # Determine match type
    if home_tier == away_tier:
        match_type = f"{home_tier} vs {away_tier}"
    else:
        tiers = sorted([home_tier, away_tier])
        match_type = f"{tiers[0]} vs {tiers[1]}"
    
    results.append({
        'season': season,
        'round': match_round,
        'home': home_team,
        'away': away_team,
        'score': match['score'],
        'goal_diff': match['goal_diff'],
        'home_position': home_position,
        'away_position': away_position,
        'home_tier': home_tier,
        'away_tier': away_tier,
        'match_type': match_type
    })

df_big_margin_classified = pd.DataFrame(results)

print(f"\n✅ Classified {len(df_big_margin_classified)} matches")
print(f"⚠️ Skipped {len(skipped)} matches")

if skipped and len(skipped) <= 10:
    print("\nSkipped matches:")
    for s in skipped:
        print(f"   {s}")

print("\n📊 Sample classified matches:")
display(df_big_margin_classified.head(10))

print("\n📊 Match type distribution:")
display(df_big_margin_classified['match_type'].value_counts())

# Save to file
output_file = PROCESSED_DIR / 'big_margin_matches_classified.csv'
df_big_margin_classified.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n💾 Saved to: {output_file}")

🔍 Classifying big margin matches by team positions...

✅ Classified 82 matches
⚠️ Skipped 437 matches

📊 Sample classified matches:


,season,round,home,away,score,goal_diff,home_position,away_position,home_tier,away_tier,match_type
0,2006/07,1,H. Kfar Saba,H. Petah Tikva,4:1,3,1,12,Top,Bottom,Bottom vs Top
1,2006/07,8,Maccabi Herzlya,M. Petah Tikva,0:3,3,11,7,Bottom,Middle,Bottom vs Middle
2,2006/07,18,Hapoel Tel Aviv,Hakoah Amidar,4:1,3,2,10,Top,Middle,Middle vs Top
3,2006/07,28,FC Ashdod,H. Kfar Saba,3:0,3,6,8,Middle,Middle,Middle vs Middle
4,2006/07,31,M. Tel Aviv,Bnei Yehuda,4:0,4,2,9,Top,Middle,Middle vs Top
5,2006/07,33,H. Kfar Saba,M. Petah Tikva,3:0,3,8,6,Middle,Middle,Middle vs Middle
6,2006/07,34,B. Jerusalem,Maccabi Herzlya,3:0,3,1,10,Top,Middle,Middle vs Top
7,2007/08,1,Kiryat Shmona,Maccabi Herzlya,3:0,3,1,11,Top,Bottom,Bottom vs Top
8,2007/08,4,B. Jerusalem,Bnei Yehuda,3:0,3,1,9,Top,Middle,Middle vs Top
9,2007/08,20,FC Ashdod,Maccabi Netanya,1:6,5,11,2,Bottom,Top,Bottom vs Top



📊 Match type distribution:


match_type
Bottom vs Top       24
Middle vs Top       20
Bottom vs Middle    19
Middle vs Middle    10
Top vs Top           6
Bottom vs Bottom     3
Name: count, dtype: int64


💾 Saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\processed\big_margin_matches_classified.csv


In [92]:
# בדיקה - למה דילגנו על כל כך הרבה משחקים?
print(f"סה\"כ משחקים בהפרש גדול: {len(df_big_margin)}")
print(f"סווגנו: {len(df_big_margin_classified)}")
print(f"דילגנו: {len(skipped)}")

if skipped:
    # ספור לפי סיבות
    from collections import Counter
    reasons = Counter([s.get('reason', 'Unknown') for s in skipped])
    print("\n📊 סיבות לדילוג:")
    for reason, count in reasons.most_common():
        print(f"   {reason}: {count}")
    
    # הצג דוגמאות
    print("\n🔍 דוגמאות של משחקים שדולגו:")
    for s in skipped[:10]:
        print(f"   {s}")

סה"כ משחקים בהפרש גדול: 519
סווגנו: 82
דילגנו: 437

📊 סיבות לדילוג:
   Round 113 not in data: 7
   Round 42 not in data: 7
   Round 109 not in data: 7
   Round 66 not in data: 6
   Round 36 not in data: 6
   Round 39 not in data: 5
   Round 127 not in data: 5
   Round 146 not in data: 5
   Round 168 not in data: 5
   Round 50 not in data: 5
   Round 75 not in data: 5
   Round 79 not in data: 5
   Round 118 not in data: 5
   Round 117 not in data: 5
   Round 131 not in data: 5
   Round 167 not in data: 5
   Round 53 not in data: 5
   Round 76 not in data: 5
   Round 149 not in data: 4
   Round 157 not in data: 4
   Round 136 not in data: 4
   Round 147 not in data: 4
   Round 44 not in data: 4
   Round 52 not in data: 4
   Round 60 not in data: 4
   Round 81 not in data: 4
   Round 150 not in data: 4
   Round 31 not in data: 4
   Round 38 not in data: 4
   Round 72 not in data: 4
   Round 73 not in data: 4
   Round 100 not in data: 4
   Round 101 not in data: 4
   Round 111 not in data:

### תיקון - התעלמות ממספרי מחזורים שגויים

הבעיה: מספרי המחזורים בקובץ המשחקים לא נכונים (כל משחק = מחזור).

הפתרון: נשתמש **רק** בנתונים שסקרפנו והתאמת קבוצות.

In [93]:
# גישה חדשה: עבור כל משחק, נמצא את המחזור שמתאים לו
# על סמך המיקומים של הקבוצות בטבלאות שסקרפנו

print("🔄 Re-classifying matches using scraped position data...")
print("=" * 70)

results = []
skipped = []
team_name_issues = set()

for _, match in df_big_margin.iterrows():
    season = match['season']
    home_team = match['home']
    away_team = match['away']
    
    # Check if we have position data for this season
    if season not in position_tables:
        skipped.append({'season': season, 'reason': 'No position data'})
        continue
    
    positions_df = position_tables[season]
    
    # Try to find this match in ANY round of the season
    # by looking for rounds where both teams exist
    found = False
    
    for round_num in positions_df.index:
        round_positions = positions_df.loc[round_num]
        
        # Check if both teams are in this round's data
        if home_team in round_positions and away_team in round_positions:
            # Found both teams - use THIS round
            home_position = int(round_positions[home_team])
            away_position = int(round_positions[away_team])
            
            # Get tiers
            home_tier = get_team_tier(home_position)
            away_tier = get_team_tier(away_position)
            
            # Determine match type
            if home_tier == away_tier:
                match_type = f"{home_tier} vs {away_tier}"
            else:
                tiers = sorted([home_tier, away_tier])
                match_type = f"{tiers[0]} vs {tiers[1]}"
            
            results.append({
                'season': season,
                'actual_round': round_num,  # המחזור האמיתי מהסקרייפינג
                'home': home_team,
                'away': away_team,
                'score': match['score'],
                'goal_diff': match['goal_diff'],
                'home_position': home_position,
                'away_position': away_position,
                'home_tier': home_tier,
                'away_tier': away_tier,
                'match_type': match_type
            })
            found = True
            break  # מצאנו, עוברים למשחק הבא
    
    if not found:
        team_name_issues.add(f"{home_team} or {away_team} in {season}")
        skipped.append({
            'season': season,
            'home': home_team,
            'away': away_team,
            'reason': 'Teams not found in any round'
        })

df_big_margin_classified = pd.DataFrame(results)

print(f"\n✅ Classified {len(df_big_margin_classified)} matches")
print(f"⚠️ Skipped {len(skipped)} matches")

if team_name_issues:
    print(f"\n⚠️ Found {len(team_name_issues)} team name issues")
    print("Sample issues:")
    for issue in list(team_name_issues)[:10]:
        print(f"   {issue}")

print("\n📊 Sample classified matches:")
display(df_big_margin_classified[['season', 'actual_round', 'home', 'away', 'score', 
                                    'home_position', 'away_position', 'match_type']].head(15))

print("\n📊 Match type distribution:")
display(df_big_margin_classified['match_type'].value_counts())

# Save to file
output_file = PROCESSED_DIR / 'big_margin_matches_classified.csv'
df_big_margin_classified.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n💾 Saved to: {output_file}")

🔄 Re-classifying matches using scraped position data...

✅ Classified 519 matches
⚠️ Skipped 0 matches

📊 Sample classified matches:


,season,actual_round,home,away,score,home_position,away_position,match_type
0,2006/07,1,H. Kfar Saba,H. Petah Tikva,4:1,1,12,Bottom vs Top
1,2006/07,1,Maccabi Herzlya,M. Petah Tikva,0:3,9,7,Middle vs Middle
2,2006/07,1,Hapoel Tel Aviv,Hakoah Amidar,4:1,5,7,Middle vs Middle
3,2006/07,1,FC Ashdod,H. Kfar Saba,3:0,4,1,Top vs Top
4,2006/07,1,M. Tel Aviv,Bnei Yehuda,4:0,11,5,Bottom vs Middle
5,2006/07,1,H. Kfar Saba,M. Petah Tikva,3:0,1,7,Middle vs Top
6,2006/07,1,B. Jerusalem,Maccabi Herzlya,3:0,3,9,Middle vs Top
7,2006/07,1,Hakoah Amidar,H. Petah Tikva,0:3,7,12,Bottom vs Middle
8,2006/07,1,FC Ashdod,B. Jerusalem,1:4,4,3,Top vs Top
9,2006/07,1,Maccabi Haifa,Maccabi Herzlya,0:4,10,9,Middle vs Middle



📊 Match type distribution:


match_type
Middle vs Top       143
Bottom vs Middle    116
Bottom vs Top       113
Middle vs Middle     66
Top vs Top           53
Bottom vs Bottom     28
Name: count, dtype: int64


💾 Saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\processed\big_margin_matches_classified.csv


### תיקון נוסף - חישוב מחזור אמיתי

הבעיה הקודמת: מצאנו את כל המשחקים במחזור 1.

גישה חדשה: נשתמש במספר המשחק **ברצף** בעונה כדי לחשב את המחזור האמיתי.

In [94]:
# גישה מתוקנת: נחשב מחזור אמיתי לפי מספר המשחק ברצף
print("🔄 Re-calculating actual rounds based on match sequence...")
print("=" * 70)

# קודם כל, נמיין את כל המשחקים לפי עונה ומספר המשחק הרציף בקובץ
df_matches_sorted = df_matches.sort_values(['season', 'round']).reset_index(drop=True)

# עבור כל עונה, נחשב את המחזור האמיתי
df_matches_sorted['actual_round'] = 0

for season in df_matches_sorted['season'].unique():
    season_mask = df_matches_sorted['season'] == season
    season_matches = df_matches_sorted[season_mask]
    
    # כמה קבוצות יש בעונה הזו?
    year = int(season.split('/')[0])
    if year <= 2008:
        num_teams = 12
    elif year <= 2011:
        num_teams = 16
    else:
        num_teams = 14
    
    # כמה משחקים במחזור? (כל קבוצה משחקת, אז זה חצי ממספר הקבוצות)
    matches_per_round = num_teams // 2
    
    # חשב מחזור אמיתי לכל משחק
    match_numbers = range(len(season_matches))
    actual_rounds = [(i // matches_per_round) + 1 for i in match_numbers]
    
    # עדכן ב-DataFrame
    df_matches_sorted.loc[season_mask, 'actual_round'] = actual_rounds
    
    print(f"   {season}: {len(season_matches)} matches, {num_teams} teams, {matches_per_round} matches/round")
    print(f"             Calculated {max(actual_rounds)} rounds")

# עכשיו נסווג את המשחקים בהפרש גדול עם המחזורים המתוקנים
df_big_margin_sorted = df_matches_sorted[df_matches_sorted['goal_diff'] >= 3].copy()

print(f"\n✅ Recalculated rounds for all matches")
print(f"   Big margin matches with actual rounds: {len(df_big_margin_sorted)}")

# הצג דוגמאות
print("\n📊 Sample matches with calculated rounds:")
display(df_big_margin_sorted[['season', 'actual_round', 'home', 'score', 'away', 'goal_diff']].head(10))

🔄 Re-calculating actual rounds based on match sequence...
   2006/07: 198 matches, 12 teams, 6 matches/round
             Calculated 33 rounds
   2007/08: 198 matches, 12 teams, 6 matches/round
             Calculated 33 rounds
   2008/09: 198 matches, 12 teams, 6 matches/round
             Calculated 33 rounds
   2009/10: 240 matches, 16 teams, 8 matches/round
             Calculated 30 rounds
   2010/11: 240 matches, 16 teams, 8 matches/round
             Calculated 30 rounds
   2011/12: 240 matches, 16 teams, 8 matches/round
             Calculated 30 rounds
   2012/13: 182 matches, 14 teams, 7 matches/round
             Calculated 26 rounds
   2013/14: 182 matches, 14 teams, 7 matches/round
             Calculated 26 rounds
   2014/15: 182 matches, 14 teams, 7 matches/round
             Calculated 26 rounds
   2015/16: 182 matches, 14 teams, 7 matches/round
             Calculated 26 rounds
   2016/17: 182 matches, 14 teams, 7 matches/round
             Calculated 26 rounds
   2017

,season,actual_round,home,score,away,goal_diff
0,2006/07,1,H. Kfar Saba,4:1,H. Petah Tikva,3
7,2006/07,2,Maccabi Herzlya,0:3,M. Petah Tikva,3
17,2006/07,3,Hapoel Tel Aviv,4:1,Hakoah Amidar,3
27,2006/07,5,FC Ashdod,3:0,H. Kfar Saba,3
30,2006/07,6,M. Tel Aviv,4:0,Bnei Yehuda,4
32,2006/07,6,H. Kfar Saba,3:0,M. Petah Tikva,3
33,2006/07,6,B. Jerusalem,3:0,Maccabi Herzlya,3
39,2006/07,7,Hakoah Amidar,0:3,H. Petah Tikva,3
40,2006/07,7,FC Ashdod,1:4,B. Jerusalem,3
102,2006/07,18,Maccabi Haifa,0:4,Maccabi Herzlya,4


In [95]:
# עכשיו נסווג עם המחזורים המתוקנים
print("🎯 Final classification with corrected rounds...")
print("=" * 70)

results = []
skipped = []

for _, match in df_big_margin_sorted.iterrows():
    season = match['season']
    actual_round = match['actual_round']
    home_team = match['home']
    away_team = match['away']
    
    # Check if we have position data for this season
    if season not in position_tables:
        skipped.append({'season': season, 'reason': 'No position data'})
        continue
    
    positions_df = position_tables[season]
    
    # Find position BEFORE this match
    # If round 1, use round 1. Otherwise use previous round
    lookup_round = 1 if actual_round == 1 else actual_round - 1
    
    # Check if this round exists
    if lookup_round not in positions_df.index:
        skipped.append({
            'season': season, 
            'round': actual_round,
            'lookup_round': lookup_round,
            'reason': f'Round {lookup_round} not in position data'
        })
        continue
    
    # Get positions for this round
    round_positions = positions_df.loc[lookup_round]
    
    # Check if both teams exist
    if home_team not in round_positions:
        skipped.append({
            'season': season,
            'round': actual_round,
            'home': home_team,
            'reason': f'Home team not found in round {lookup_round}'
        })
        continue
    
    if away_team not in round_positions:
        skipped.append({
            'season': season,
            'round': actual_round,
            'away': away_team,
            'reason': f'Away team not found in round {lookup_round}'
        })
        continue
    
    # Get positions
    home_position = int(round_positions[home_team])
    away_position = int(round_positions[away_team])
    
    # Get tiers
    home_tier = get_team_tier(home_position)
    away_tier = get_team_tier(away_position)
    
    # Determine match type
    if home_tier == away_tier:
        match_type = f"{home_tier} vs {away_tier}"
    else:
        tiers = sorted([home_tier, away_tier])
        match_type = f"{tiers[0]} vs {tiers[1]}"
    
    results.append({
        'season': season,
        'round': actual_round,
        'home': home_team,
        'away': away_team,
        'score': match['score'],
        'goal_diff': match['goal_diff'],
        'home_position': home_position,
        'away_position': away_position,
        'home_tier': home_tier,
        'away_tier': away_tier,
        'match_type': match_type
    })

df_big_margin_classified = pd.DataFrame(results)

print(f"\n✅ Classified {len(df_big_margin_classified)} matches")
print(f"⚠️ Skipped {len(skipped)} matches")

if skipped:
    print(f"\nSample skipped (first 5):")
    for s in skipped[:5]:
        print(f"   {s}")

print("\n📊 Sample classified matches:")
display(df_big_margin_classified[['season', 'round', 'home', 'away', 'score', 
                                    'home_position', 'away_position', 'match_type']].head(15))

print("\n📊 Match type distribution:")
display(df_big_margin_classified['match_type'].value_counts())

print("\n📊 Distribution by round:")
print(df_big_margin_classified['round'].describe())

# Save to file
output_file = PROCESSED_DIR / 'big_margin_matches_classified.csv'
df_big_margin_classified.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n💾 Saved to: {output_file}")

🎯 Final classification with corrected rounds...

✅ Classified 519 matches
⚠️ Skipped 0 matches

📊 Sample classified matches:


,season,round,home,away,score,home_position,away_position,match_type
0,2006/07,1,H. Kfar Saba,H. Petah Tikva,4:1,1,12,Bottom vs Top
1,2006/07,2,Maccabi Herzlya,M. Petah Tikva,0:3,9,7,Middle vs Middle
2,2006/07,3,Hapoel Tel Aviv,Hakoah Amidar,4:1,9,5,Middle vs Middle
3,2006/07,5,FC Ashdod,H. Kfar Saba,3:0,6,3,Middle vs Top
4,2006/07,6,M. Tel Aviv,Bnei Yehuda,4:0,11,8,Bottom vs Middle
5,2006/07,6,H. Kfar Saba,M. Petah Tikva,3:0,7,1,Middle vs Top
6,2006/07,6,B. Jerusalem,Maccabi Herzlya,3:0,2,10,Middle vs Top
7,2006/07,7,Hakoah Amidar,H. Petah Tikva,0:3,9,12,Bottom vs Middle
8,2006/07,7,FC Ashdod,B. Jerusalem,1:4,2,1,Top vs Top
9,2006/07,18,Maccabi Haifa,Maccabi Herzlya,0:4,5,11,Bottom vs Middle



📊 Match type distribution:


match_type
Middle vs Top       159
Bottom vs Middle    115
Bottom vs Top       110
Middle vs Middle     68
Top vs Top           42
Bottom vs Bottom     25
Name: count, dtype: int64


📊 Distribution by round:
count    519.000000
mean      13.448940
std        8.067811
min        1.000000
25%        6.000000
50%       13.000000
75%       20.000000
max       33.000000
Name: round, dtype: float64

💾 Saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\processed\big_margin_matches_classified.csv


## 7. ניתוח משחקי תיקו

נבדוק כמה משחקים הסתיימו בתיקו בכל עונה.

In [96]:
# זיהוי משחקי תיקו (home_goals == away_goals)
df_draws = df_matches_sorted[df_matches_sorted['home_goals'] == df_matches_sorted['away_goals']].copy()

print("⚖️ Draw Analysis")
print("=" * 70)
print(f"Total draws: {len(df_draws)} matches")
print(f"Percentage: {len(df_draws) / len(df_matches_sorted) * 100:.2f}% of all matches")

# ניתוח לפי עונה
print("\n📊 Draws by season:")
draws_by_season = df_draws.groupby('season').size().to_frame('draws')
total_by_season = df_matches_sorted.groupby('season').size().to_frame('total_matches')
summary = draws_by_season.join(total_by_season)
summary['draw_pct'] = (summary['draws'] / summary['total_matches'] * 100).round(2)
summary = summary.sort_index()

display(summary)

# סטטיסטיקה כללית
print("\n📈 Draw statistics:")
print(f"   Average draws per season: {summary['draws'].mean():.1f}")
print(f"   Average draw percentage: {summary['draw_pct'].mean():.2f}%")
print(f"   Highest draw season: {summary['draw_pct'].idxmax()} ({summary['draw_pct'].max():.2f}%)")
print(f"   Lowest draw season: {summary['draw_pct'].idxmin()} ({summary['draw_pct'].min():.2f}%)")

# התפלגות תוצאות תיקו
print("\n⚖️ Draw score distribution:")
df_draws['draw_score'] = df_draws['home_goals'].astype(int).astype(str) + ':' + df_draws['away_goals'].astype(int).astype(str)
draw_scores = df_draws['draw_score'].value_counts().head(10)
display(draw_scores.to_frame('Count'))

# שמירה לקובץ
output_file = PROCESSED_DIR / 'draws_by_season.csv'
summary.to_csv(output_file, encoding='utf-8-sig')
print(f"\n💾 Saved to: {output_file}")

⚖️ Draw Analysis
Total draws: 1034 matches
Percentage: 27.54% of all matches

📊 Draws by season:


,draws,total_matches,draw_pct
season,,,
2006/07,61,198,30.81
2007/08,48,198,24.24
2008/09,62,198,31.31
2009/10,69,240,28.75
2010/11,64,240,26.67
2011/12,67,240,27.92
2012/13,50,182,27.47
2013/14,46,182,25.27
2014/15,52,182,28.57



📈 Draw statistics:
   Average draws per season: 51.7
   Average draw percentage: 27.46%
   Highest draw season: 2018/19 (33.52%)
   Lowest draw season: 2019/20 (24.18%)

⚖️ Draw score distribution:


,Count
draw_score,
1:1,453
0:0,372
2:2,175
3:3,29
4:4,5



💾 Saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\processed\draws_by_season.csv


## 8. ניתוח הפתעות: תחתית מול צמרת

נבדק מי ניצח במשחקי הפער הגדולים בין קבוצות תחתית לקבוצות צמרת:
- 🔴 **צמרת ניצחה תחתית בתוצאה גבוהה** - צפוי, מראה על פער גדול
- 🟢 **תחתית ניצחה צמרת בתוצאה גבוהה** - הפתעה גדולה, מראה על תחרותיות!

In [100]:
# נשתמש ב-DataFrame המסווג שכבר יש לנו בזיכרון
# זיהוי משחקי Top vs Bottom בלבד
top_vs_bottom = df_big_margin_classified[
    ((df_big_margin_classified['home_tier'] == 'Top') & (df_big_margin_classified['away_tier'] == 'Bottom')) |
    ((df_big_margin_classified['home_tier'] == 'Bottom') & (df_big_margin_classified['away_tier'] == 'Top'))
].copy()

print("🎯 Top vs Bottom Big Margin Matches Analysis")
print("=" * 70)
print(f"Total Top vs Bottom matches: {len(top_vs_bottom)}")

# פירוק התוצאה למספרי שערים
top_vs_bottom[['home_goals', 'away_goals']] = top_vs_bottom['score'].str.split(':', expand=True).astype(int)

# הוספת עמודה למי ניצח
def determine_winner(row):
    if row['home_goals'] > row['away_goals']:
        # הבית ניצח
        if row['home_tier'] == 'Top':
            return 'Top won (expected)'
        else:
            return 'Bottom won (upset!)'
    else:
        # החוץ ניצח
        if row['away_tier'] == 'Top':
            return 'Top won (expected)'
        else:
            return 'Bottom won (upset!)'

top_vs_bottom['winner_category'] = top_vs_bottom.apply(determine_winner, axis=1)

# ספירה לפי קטגוריה
winner_counts = top_vs_bottom['winner_category'].value_counts()
print("\n📊 Results breakdown:")
display(winner_counts.to_frame('Count'))

# אחוזים
total = len(top_vs_bottom)
print("\n📈 Percentages:")
for category, count in winner_counts.items():
    pct = (count / total * 100)
    icon = "🔴" if "expected" in category else "🟢"
    print(f"   {icon} {category}: {count} ({pct:.1f}%)")

# ניתוח לפי עונה
print("\n\n📅 Upsets by season:")
upsets_by_season = top_vs_bottom[top_vs_bottom['winner_category'] == 'Bottom won (upset!)'].groupby('season').size()
expected_by_season = top_vs_bottom[top_vs_bottom['winner_category'] == 'Top won (expected)'].groupby('season').size()

season_analysis = pd.DataFrame({
    'upsets': upsets_by_season,
    'expected': expected_by_season
}).fillna(0).astype(int)

season_analysis['total'] = season_analysis['upsets'] + season_analysis['expected']
season_analysis['upset_pct'] = (season_analysis['upsets'] / season_analysis['total'] * 100).round(2)
season_analysis = season_analysis.sort_index()

display(season_analysis)

# סטטיסטיקה
print("\n🏆 Upset statistics:")
print(f"   Average upsets per season: {season_analysis['upsets'].mean():.1f}")
print(f"   Average upset percentage: {season_analysis['upset_pct'].mean():.2f}%")
print(f"   Highest upset season: {season_analysis['upset_pct'].idxmax()} ({season_analysis['upset_pct'].max():.2f}%)")
print(f"   Lowest upset season: {season_analysis['upset_pct'].idxmin()} ({season_analysis['upset_pct'].min():.2f}%)")

# שמירה
top_vs_bottom.to_csv(PROCESSED_DIR / 'top_vs_bottom_detailed.csv', index=False, encoding='utf-8-sig')
season_analysis.to_csv(PROCESSED_DIR / 'upsets_by_season.csv', encoding='utf-8-sig')
print(f"\n💾 Saved detailed results")

🎯 Top vs Bottom Big Margin Matches Analysis
Total Top vs Bottom matches: 110

📊 Results breakdown:


,Count
winner_category,
Top won (expected),101
Bottom won (upset!),9



📈 Percentages:
   🔴 Top won (expected): 101 (91.8%)
   🟢 Bottom won (upset!): 9 (8.2%)


📅 Upsets by season:


,upsets,expected,total,upset_pct
season,,,,
2006/07,0,2,2,0.00
2007/08,0,2,2,0.00
2008/09,0,3,3,0.00
2009/10,1,14,15,6.67
2010/11,1,7,8,12.50
2011/12,1,4,5,20.00
2012/13,0,6,6,0.00
2013/14,0,4,4,0.00
2014/15,0,8,8,0.00



🏆 Upset statistics:
   Average upsets per season: 0.5
   Average upset percentage: 7.05%
   Highest upset season: 2021/22 (50.00%)
   Lowest upset season: 2006/07 (0.00%)

💾 Saved detailed results


## 9. ניתוח הפתעות: אמצע מול צמרת

נבדק מי ניצח במשחקי הפער הגדולים בין קבוצות אמצע לקבוצות צמרת:
- 🔴 **צמרת ניצחה אמצע בתוצאה גבוהה** - צפוי, מראה על פער
- 🟢 **אמצע ניצחה צמרת בתוצאה גבוהה** - הפתעה, מראה על תחרותיות!

In [101]:
# זיהוי משחקי Top vs Middle בלבד
top_vs_middle = df_big_margin_classified[
    ((df_big_margin_classified['home_tier'] == 'Top') & (df_big_margin_classified['away_tier'] == 'Middle')) |
    ((df_big_margin_classified['home_tier'] == 'Middle') & (df_big_margin_classified['away_tier'] == 'Top'))
].copy()

print("🎯 Top vs Middle Big Margin Matches Analysis")
print("=" * 70)
print(f"Total Top vs Middle matches: {len(top_vs_middle)}")

# פירוק התוצאה למספרי שערים
top_vs_middle[['home_goals', 'away_goals']] = top_vs_middle['score'].str.split(':', expand=True).astype(int)

# הוספת עמודה למי ניצח
def determine_winner_middle(row):
    if row['home_goals'] > row['away_goals']:
        # הבית ניצח
        if row['home_tier'] == 'Top':
            return 'Top won (expected)'
        else:
            return 'Middle won (upset!)'
    else:
        # החוץ ניצח
        if row['away_tier'] == 'Top':
            return 'Top won (expected)'
        else:
            return 'Middle won (upset!)'

top_vs_middle['winner_category'] = top_vs_middle.apply(determine_winner_middle, axis=1)

# ספירה לפי קטגוריה
winner_counts = top_vs_middle['winner_category'].value_counts()
print("\n📊 Results breakdown:")
display(winner_counts.to_frame('Count'))

# אחוזים
total = len(top_vs_middle)
print("\n📈 Percentages:")
for category, count in winner_counts.items():
    pct = (count / total * 100)
    icon = "🔴" if "expected" in category else "🟢"
    print(f"   {icon} {category}: {count} ({pct:.1f}%)")

# ניתוח לפי עונה
print("\n\n📅 Upsets by season:")
upsets_by_season = top_vs_middle[top_vs_middle['winner_category'] == 'Middle won (upset!)'].groupby('season').size()
expected_by_season = top_vs_middle[top_vs_middle['winner_category'] == 'Top won (expected)'].groupby('season').size()

season_analysis_middle = pd.DataFrame({
    'upsets': upsets_by_season,
    'expected': expected_by_season
}).fillna(0).astype(int)

season_analysis_middle['total'] = season_analysis_middle['upsets'] + season_analysis_middle['expected']
season_analysis_middle['upset_pct'] = (season_analysis_middle['upsets'] / season_analysis_middle['total'] * 100).round(2)
season_analysis_middle = season_analysis_middle.sort_index()

display(season_analysis_middle)

# סטטיסטיקה
print("\n🏆 Upset statistics:")
print(f"   Average upsets per season: {season_analysis_middle['upsets'].mean():.1f}")
print(f"   Average upset percentage: {season_analysis_middle['upset_pct'].mean():.2f}%")
print(f"   Highest upset season: {season_analysis_middle['upset_pct'].idxmax()} ({season_analysis_middle['upset_pct'].max():.2f}%)")
print(f"   Lowest upset season: {season_analysis_middle['upset_pct'].idxmin()} ({season_analysis_middle['upset_pct'].min():.2f}%)")

# שמירה
top_vs_middle.to_csv(PROCESSED_DIR / 'top_vs_middle_detailed.csv', index=False, encoding='utf-8-sig')
season_analysis_middle.to_csv(PROCESSED_DIR / 'upsets_middle_by_season.csv', encoding='utf-8-sig')
print(f"\n💾 Saved detailed results")

🎯 Top vs Middle Big Margin Matches Analysis
Total Top vs Middle matches: 159

📊 Results breakdown:


,Count
winner_category,
Top won (expected),131
Middle won (upset!),28



📈 Percentages:
   🔴 Top won (expected): 131 (82.4%)
   🟢 Middle won (upset!): 28 (17.6%)


📅 Upsets by season:


,upsets,expected,total,upset_pct
season,,,,
2006/07,4,5,9,44.44
2007/08,2,10,12,16.67
2008/09,2,4,6,33.33
2009/10,0,7,7,0.00
2010/11,2,7,9,22.22
2011/12,2,2,4,50.00
2012/13,2,6,8,25.00
2013/14,0,6,6,0.00
2014/15,2,5,7,28.57



🏆 Upset statistics:
   Average upsets per season: 1.4
   Average upset percentage: 17.15%
   Highest upset season: 2011/12 (50.00%)
   Lowest upset season: 2009/10 (0.00%)

💾 Saved detailed results


## 10. ניתוח הפתעות: תחתית מול אמצע

נבדק מי ניצח במשחקי הפער הגדולים בין קבוצות תחתית לקבוצות אמצע:
- 🔴 **אמצע ניצחה תחתית בתוצאה גבוהה** - צפוי, מראה על פער
- 🟢 **תחתית ניצחה אמצע בתוצאה גבוהה** - הפתעה, מראה על תחרותיות!

In [102]:
# זיהוי משחקי Middle vs Bottom בלבד
middle_vs_bottom = df_big_margin_classified[
    ((df_big_margin_classified['home_tier'] == 'Middle') & (df_big_margin_classified['away_tier'] == 'Bottom')) |
    ((df_big_margin_classified['home_tier'] == 'Bottom') & (df_big_margin_classified['away_tier'] == 'Middle'))
].copy()

print("🎯 Middle vs Bottom Big Margin Matches Analysis")
print("=" * 70)
print(f"Total Middle vs Bottom matches: {len(middle_vs_bottom)}")

# פירוק התוצאה למספרי שערים
middle_vs_bottom[['home_goals', 'away_goals']] = middle_vs_bottom['score'].str.split(':', expand=True).astype(int)

# הוספת עמודה למי ניצח
def determine_winner_bottom(row):
    if row['home_goals'] > row['away_goals']:
        # הבית ניצח
        if row['home_tier'] == 'Middle':
            return 'Middle won (expected)'
        else:
            return 'Bottom won (upset!)'
    else:
        # החוץ ניצח
        if row['away_tier'] == 'Middle':
            return 'Middle won (expected)'
        else:
            return 'Bottom won (upset!)'

middle_vs_bottom['winner_category'] = middle_vs_bottom.apply(determine_winner_bottom, axis=1)

# ספירה לפי קטגוריה
winner_counts = middle_vs_bottom['winner_category'].value_counts()
print("\n📊 Results breakdown:")
display(winner_counts.to_frame('Count'))

# אחוזים
total = len(middle_vs_bottom)
print("\n📈 Percentages:")
for category, count in winner_counts.items():
    pct = (count / total * 100)
    icon = "🔴" if "expected" in category else "🟢"
    print(f"   {icon} {category}: {count} ({pct:.1f}%)")

# ניתוח לפי עונה
print("\n\n📅 Upsets by season:")
upsets_by_season = middle_vs_bottom[middle_vs_bottom['winner_category'] == 'Bottom won (upset!)'].groupby('season').size()
expected_by_season = middle_vs_bottom[middle_vs_bottom['winner_category'] == 'Middle won (expected)'].groupby('season').size()

season_analysis_bottom = pd.DataFrame({
    'upsets': upsets_by_season,
    'expected': expected_by_season
}).fillna(0).astype(int)

season_analysis_bottom['total'] = season_analysis_bottom['upsets'] + season_analysis_bottom['expected']
season_analysis_bottom['upset_pct'] = (season_analysis_bottom['upsets'] / season_analysis_bottom['total'] * 100).round(2)
season_analysis_bottom = season_analysis_bottom.sort_index()

display(season_analysis_bottom)

# סטטיסטיקה
print("\n🏆 Upset statistics:")
print(f"   Average upsets per season: {season_analysis_bottom['upsets'].mean():.1f}")
print(f"   Average upset percentage: {season_analysis_bottom['upset_pct'].mean():.2f}%")
print(f"   Highest upset season: {season_analysis_bottom['upset_pct'].idxmax()} ({season_analysis_bottom['upset_pct'].max():.2f}%)")
print(f"   Lowest upset season: {season_analysis_bottom['upset_pct'].idxmin()} ({season_analysis_bottom['upset_pct'].min():.2f}%)")

# שמירה
middle_vs_bottom.to_csv(PROCESSED_DIR / 'middle_vs_bottom_detailed.csv', index=False, encoding='utf-8-sig')
season_analysis_bottom.to_csv(PROCESSED_DIR / 'upsets_bottom_by_season.csv', encoding='utf-8-sig')
print(f"\n💾 Saved detailed results")

🎯 Middle vs Bottom Big Margin Matches Analysis
Total Middle vs Bottom matches: 115

📊 Results breakdown:


,Count
winner_category,
Middle won (expected),80
Bottom won (upset!),35



📈 Percentages:
   🔴 Middle won (expected): 80 (69.6%)
   🟢 Bottom won (upset!): 35 (30.4%)


📅 Upsets by season:


,upsets,expected,total,upset_pct
season,,,,
2006/07,4,4,8,50.00
2007/08,2,2,4,50.00
2008/09,0,1,1,0.00
2009/10,1,7,8,12.50
2010/11,1,9,10,10.00
2011/12,1,14,15,6.67
2012/13,4,2,6,66.67
2013/14,0,4,4,0.00
2014/15,1,2,3,33.33



🏆 Upset statistics:
   Average upsets per season: 1.8
   Average upset percentage: 29.58%
   Highest upset season: 2012/13 (66.67%)
   Lowest upset season: 2008/09 (0.00%)

💾 Saved detailed results


## 11. סיכום תחרותיות לפי עונות - טבלה מאוחדת

נאחד את כל הניתוחים לטבלה אחת שמראה את רמת התחרותיות בכל עונה:
- מספר קבוצות בליגה
- אחוז תיקו
- משחקי תוצאה גבוהה (3+ שערים הפרש)
- אחוז הפתעות בכל קטגוריית משחק

In [103]:
# בניית טבלת סיכום מקיפה לכל עונה
print("🏆 Building Comprehensive Competitiveness Table by Season")
print("=" * 80)

# 1. מספר קבוצות לפי עונה
teams_per_season = df_matches_sorted.groupby('season').apply(
    lambda x: len(set(x['home'].unique()) | set(x['away'].unique()))
).to_frame('num_teams')

# 2. סך משחקים לעונה
total_matches = df_matches_sorted.groupby('season').size().to_frame('total_matches')

# 3. אחוז תיקו (מהניתוח הקודם)
draws_summary = pd.read_csv(PROCESSED_DIR / 'draws_by_season.csv', index_col='season')

# 4. משחקי תוצאה גבוהה
big_margin_count = df_big_margin_classified.groupby('season').size().to_frame('big_margin_matches')
big_margin_pct = (big_margin_count['big_margin_matches'] / total_matches['total_matches'] * 100).round(2).to_frame('big_margin_pct')

# 5. הפתעות תחתית vs צמרת
upsets_bottom_top = pd.read_csv(PROCESSED_DIR / 'upsets_by_season.csv', index_col='season')
upsets_bottom_top = upsets_bottom_top.rename(columns={'upset_pct': 'upsets_bottom_vs_top_pct'})

# 6. הפתעות אמצע vs צמרת
upsets_middle_top = pd.read_csv(PROCESSED_DIR / 'upsets_middle_by_season.csv', index_col='season')
upsets_middle_top = upsets_middle_top.rename(columns={'upset_pct': 'upsets_middle_vs_top_pct'})

# 7. הפתעות תחתית vs אמצע
upsets_bottom_middle = pd.read_csv(PROCESSED_DIR / 'upsets_bottom_by_season.csv', index_col='season')
upsets_bottom_middle = upsets_bottom_middle.rename(columns={'upset_pct': 'upsets_bottom_vs_middle_pct'})

# איחוד כל הנתונים
competitiveness_table = teams_per_season.copy()
competitiveness_table = competitiveness_table.join(total_matches)
competitiveness_table = competitiveness_table.join(draws_summary[['draws', 'draw_pct']])
competitiveness_table = competitiveness_table.join(big_margin_count)
competitiveness_table = competitiveness_table.join(big_margin_pct)
competitiveness_table = competitiveness_table.join(upsets_bottom_top[['upsets_bottom_vs_top_pct']])
competitiveness_table = competitiveness_table.join(upsets_middle_top[['upsets_middle_vs_top_pct']])
competitiveness_table = competitiveness_table.join(upsets_bottom_middle[['upsets_bottom_vs_middle_pct']])

# מילוי ערכים חסרים (עונות ללא הפתעות) ב-0
competitiveness_table = competitiveness_table.fillna(0)

# חישוב ציון תחרותיות משוקלל (ככל שהציון גבוה יותר = יותר תחרותי)
# הגיון: 
# - תיקו גבוה = תחרותי (קבוצות שוות)
# - משחקי תוצאה גבוהה נמוך = תחרותי (פחות מכות)
# - הפתעות גבוה = תחרותי (חלשים מנצחים חזקים)

competitiveness_table['competitiveness_score'] = (
    (competitiveness_table['draw_pct'] * 1.0) +  # תיקו = טוב
    (100 - competitiveness_table['big_margin_pct']) * 0.5 +  # פחות מכות = טוב
    (competitiveness_table['upsets_bottom_vs_top_pct'] * 2.0) +  # הפתעות תחתית-צמרת = מצוין
    (competitiveness_table['upsets_middle_vs_top_pct'] * 1.5) +  # הפתעות אמצע-צמרת = טוב
    (competitiveness_table['upsets_bottom_vs_middle_pct'] * 1.0)  # הפתעות תחתית-אמצע = רלוונטי
).round(2)

# סידור עמודות
column_order = [
    'num_teams', 'total_matches', 'draws', 'draw_pct',
    'big_margin_matches', 'big_margin_pct',
    'upsets_bottom_vs_top_pct', 'upsets_middle_vs_top_pct', 'upsets_bottom_vs_middle_pct',
    'competitiveness_score'
]
competitiveness_table = competitiveness_table[column_order]

# שינוי שמות עמודות לעברית בתצוגה
display_table = competitiveness_table.copy()
display_table.columns = [
    'מספר קבוצות', 'סך משחקים', 'תיקו', '% תיקו',
    'משחקי פער גדול', '% פער גדול',
    '% הפתעות תחתית-צמרת', '% הפתעות אמצע-צמרת', '% הפתעות תחתית-אמצע',
    'ציון תחרותיות'
]

print("\n📊 Competitiveness Table (sorted by season):")
display(display_table)

# מיון לפי ציון תחרותיות
print("\n\n🏅 Top 5 Most Competitive Seasons:")
top_competitive = display_table.nlargest(5, 'ציון תחרותיות')
display(top_competitive)

print("\n\n⚠️ Top 5 Least Competitive Seasons:")
least_competitive = display_table.nsmallest(5, 'ציון תחרותיות')
display(least_competitive)

# ניתוח לפי מספר קבוצות
print("\n\n📈 Average Competitiveness by Number of Teams:")
by_team_count = competitiveness_table.groupby('num_teams').agg({
    'draw_pct': 'mean',
    'big_margin_pct': 'mean',
    'upsets_bottom_vs_top_pct': 'mean',
    'upsets_middle_vs_top_pct': 'mean',
    'upsets_bottom_vs_middle_pct': 'mean',
    'competitiveness_score': 'mean'
}).round(2)

by_team_count.columns = ['ממוצע % תיקו', 'ממוצע % פער גדול', 
                          'ממוצע % הפתעות תחתית-צמרת', 'ממוצע % הפתעות אמצע-צמרת',
                          'ממוצע % הפתעות תחתית-אמצע', 'ממוצע ציון תחרותיות']
by_team_count.index.name = 'מספר קבוצות'
display(by_team_count)

# שמירה
competitiveness_table.to_csv(PROCESSED_DIR / 'competitiveness_by_season.csv', encoding='utf-8-sig')
by_team_count.to_csv(PROCESSED_DIR / 'competitiveness_by_team_count.csv', encoding='utf-8-sig')
print("\n💾 Saved: competitiveness_by_season.csv & competitiveness_by_team_count.csv")

🏆 Building Comprehensive Competitiveness Table by Season

📊 Competitiveness Table (sorted by season):


,מספר קבוצות,סך משחקים,תיקו,% תיקו,משחקי פער גדול,% פער גדול,% הפתעות תחתית-צמרת,% הפתעות אמצע-צמרת,% הפתעות תחתית-אמצע,ציון תחרותיות
season,,,,,,,,,,
2006/07,12,198,61,30.81,23,11.62,0.00,44.44,50.00,191.66
2007/08,12,198,48,24.24,26,13.13,0.00,16.67,50.00,142.68
2008/09,12,198,62,31.31,17,8.59,0.00,33.33,0.00,127.01
2009/10,16,240,69,28.75,38,15.83,6.67,0.00,12.50,96.68
2010/11,16,240,64,26.67,37,15.42,12.50,22.22,10.00,137.29
2011/12,16,240,67,27.92,34,14.17,20.00,50.00,6.67,192.50
2012/13,14,182,50,27.47,27,14.84,0.00,25.00,66.67,174.22
2013/14,14,182,46,25.27,21,11.54,0.00,0.00,0.00,69.50
2014/15,14,182,52,28.57,23,12.64,0.00,28.57,33.33,148.44




🏅 Top 5 Most Competitive Seasons:


,מספר קבוצות,סך משחקים,תיקו,% תיקו,משחקי פער גדול,% פער גדול,% הפתעות תחתית-צמרת,% הפתעות אמצע-צמרת,% הפתעות תחתית-אמצע,ציון תחרותיות
season,,,,,,,,,,
2021/22,14,182,47,25.82,22,12.09,50.0,0.00,42.86,212.64
2018/19,14,182,61,33.52,25,13.74,37.5,0.00,60.00,211.65
2011/12,16,240,67,27.92,34,14.17,20.0,50.00,6.67,192.50
2006/07,12,198,61,30.81,23,11.62,0.0,44.44,50.00,191.66
2012/13,14,182,50,27.47,27,14.84,0.0,25.00,66.67,174.22




⚠️ Top 5 Least Competitive Seasons:


,מספר קבוצות,סך משחקים,תיקו,% תיקו,משחקי פער גדול,% פער גדול,% הפתעות תחתית-צמרת,% הפתעות אמצע-צמרת,% הפתעות תחתית-אמצע,ציון תחרותיות
season,,,,,,,,,,
2025/26,14,74,19,25.68,13,17.57,0.00,0.0,0.0,66.90
2017/18,14,182,45,24.73,22,12.09,0.00,0.0,0.0,68.68
2013/14,14,182,46,25.27,21,11.54,0.00,0.0,0.0,69.50
2009/10,16,240,69,28.75,38,15.83,6.67,0.0,12.5,96.68
2016/17,14,182,57,31.32,22,12.09,0.00,20.0,0.0,105.28




📈 Average Competitiveness by Number of Teams:


,ממוצע % תיקו,ממוצע % פער גדול,ממוצע % הפתעות תחתית-צמרת,ממוצע % הפתעות אמצע-צמרת,ממוצע % הפתעות תחתית-אמצע,ממוצע ציון תחרותיות
מספר קבוצות,,,,,,
12,28.79,11.11,0.00,31.48,33.33,153.78
14,27.11,14.25,7.27,12.60,33.03,136.45
16,27.78,15.14,13.06,24.07,9.72,142.16



💾 Saved: competitiveness_by_season.csv & competitiveness_by_team_count.csv
